# 🌎 Welcome to the CSE151B Spring 2025 Climate Emulation Competition!

Thank you for participating in this exciting challenge focused on building machine learning models to emulate complex climate systems.  
This notebook is provided as a **starter template** to help you:

- Understand how to load and preprocess the dataset  
- Construct a baseline model  
- Train and evaluate predictions using a PyTorch Lightning pipeline  
- Format your predictions for submission to the leaderboard  

You're encouraged to:
- Build on this structure or replace it entirely
- Try more advanced models and training strategies
- Incorporate your own ideas to push the boundaries of what's possible

If you're interested in developing within a repository structure and/or use helpful tools like configuration management (based on Hydra) and logging (with Weights & Biases), we recommend checking out the following Github repo. Such a structure can be useful when running multiple experiments and trying various research ideas.

👉 [https://github.com/salvaRC/cse151b-spring2025-competition](https://github.com/salvaRC/cse151b-spring2025-competition)

Good luck, have fun, and we hope you learn a lot through this process!


### 📦 Install Required Libraries
We install the necessary Python packages for data loading, deep learning, and visualization.


In [ ]:
!pip install xarray zarr dask lightning matplotlib wandb cftime einops xr --quiet

In [1]:
import os
from datetime import datetime
import numpy as np
import xarray as xr
import dask.array as da
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import lightning.pytorch as pl
import math
import torch.nn.functional as F
from torchvision import transforms

### ⚙️ Configuration Setup  
Define all model, data, and training hyperparameters in one place for easy control and reproducibility.

### 📊 Data Configuration

We define the dataset settings used for training and evaluation. This includes:

- **`path`**: Path to the `.zarr` dataset containing monthly climate variables from CMIP6 simulations.
- **`input_vars`**: Climate forcing variables (e.g., CO₂, CH₄) used as model inputs.
- **`output_vars`**: Target variables to predict — surface air temperature (`tas`) and precipitation (`pr`).
- **`target_member_id`**: Ensemble member to use from the simulations (each SSP has 3) for target variables.
- **`train_ssps`**: SSP scenarios used for training (low to high emissions).
- **`test_ssp`**: Scenario held out for evaluation (Must be set to SSP245).
- **`test_months`**: Number of months to include in the test split (Must be set to 120).
- **`batch_size`** and **`num_workers`**: Data loading parameters for PyTorch training.

These settings reflect how the challenge is structured: models must learn from some emission scenarios and generalize to unseen ones.

> ⚠️ **Important:** Do **not modify** the following test settings:
>
> - `test_ssp` must remain **`ssp245`**, which is the held-out evaluation scenario.
> - `test_months` must be **`120`**, corresponding to the last 10 years (monthly resolution) of the scenario.



In [30]:
#NOTE Change the data directory according to where you have your zarr files stored
config = {
    "data": {
        "path": "/kaggle/input/cse151b-spring2025-competition/processed_data_cse151b_v2_corrupted_ssp245/processed_data_cse151b_v2_corrupted_ssp245.zarr",
        "input_vars": ["CO2", "SO2", "CH4", "BC", "rsdt"],
        "output_vars": ["tas", "pr"],
        "target_member_id": [0,1,2],
        "train_ssps": ["ssp126", "ssp370", "ssp585"],
        "test_ssp": "ssp245",
        "test_months": 360,
        "batch_size": 16,
        "num_workers": 4,
        "seq_len": 12,
        "predict_seq": False,
    },
    "model": {
        "type": "simple_cnn",
        "kernel_size": 3,
        "init_dim": 64,
        "depth": 4,
        "dropout_rate": 0.1,
    },
    "training": {
        "lr": 1e-3,
    },
    "trainer": {
        "max_epochs": 20,
        "accelerator": "gpu",
        "devices": "auto",
        "precision": 32,
        "deterministic": True,
        "num_sanity_val_steps": 0,
    },
    "seed": 42,
}
pl.seed_everything(config["seed"])  # Set seed for reproducibility

INFO: Seed set to 42


42

### 🔧 Spatial Weighting Utility Function

This cell sets up utility functions for reproducibility and spatial weighting:

- **`get_lat_weights(latitude_values)`**: Computes cosine-based area weights for each latitude, accounting for the Earth's curvature. This is critical for evaluating global climate metrics fairly — grid cells near the equator represent larger surface areas than those near the poles.


In [3]:
def get_lat_weights(latitude_values):
    lat_rad = np.deg2rad(latitude_values)
    weights = np.cos(lat_rad)
    return weights / np.mean(weights)

### 🧠 SimpleCNN: A Residual Convolutional Baseline

This is a lightweight baseline model designed to capture spatial patterns in global climate data using convolutional layers.

- The architecture starts with a **convolution + batch norm + ReLU** block to process the input channels.
- It then applies a series of **residual blocks** to extract increasingly abstract spatial features. These help preserve gradient flow during training.
- Finally, a few convolutional layers reduce the feature maps down to the desired number of output channels (`tas` and `pr`).

This model only serves as a **simple baseline for climate emulation**. 

We encourage you to build and experiment with your own models and ideas.


In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=kernel_size // 2)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, padding=kernel_size // 2)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride), nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.skip(identity)
        return self.relu(out)

class SimpleCNN(nn.Module):
    def __init__(self, n_input_channels, n_output_channels, kernel_size=3, init_dim=64, depth=4, dropout_rate=0.2):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(n_input_channels, init_dim, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.BatchNorm2d(init_dim),
            nn.ReLU(inplace=True),
        )
        self.res_blocks = nn.ModuleList()
        current_dim = init_dim
        for i in range(depth):
            out_dim = current_dim * 2 if i < depth - 1 else current_dim
            self.res_blocks.append(ResidualBlock(current_dim, out_dim))
            if i < depth - 1:
                current_dim *= 2
        self.dropout = nn.Dropout2d(dropout_rate)
        self.final = nn.Sequential(
            nn.Conv2d(current_dim, current_dim // 2, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.BatchNorm2d(current_dim // 2),
            nn.ReLU(inplace=True),
            nn.Conv2d(current_dim // 2, n_output_channels, kernel_size=1),
        )

    def forward(self, x):
        x = self.initial(x)
        for res_block in self.res_blocks:
            x = res_block(x)
        return self.final(self.dropout(x))


### 📐 Normalizer: Z-Score Scaling for Climate Inputs & Outputs

This class handles **Z-score normalization**, a crucial preprocessing step for stable and efficient neural network training:

- **`set_input_statistics(mean, std)` / `set_output_statistics(...)`**: Store the mean and standard deviation computed from the training data for later use.
- **`normalize(data, data_type)`**: Standardizes the data using `(x - mean) / std`. This is applied separately to inputs and outputs.
- **`inverse_transform_output(data)`**: Converts model predictions back to the original physical units (e.g., Kelvin for temperature, mm/day for precipitation).

Normalizing the data ensures the model sees inputs with similar dynamic ranges and avoids biases caused by different variable scales.


In [5]:
class Normalizer:
    def __init__(self):
        self.mean_in, self.std_in = None, None
        self.mean_out, self.std_out = None, None

    def set_input_statistics(self, mean, std):
        self.mean_in = mean
        self.std_in = std

    def set_output_statistics(self, mean, std):
        self.mean_out = mean
        self.std_out = std

    def normalize(self, data, data_type):
        if data_type == "input":
            return (data - self.mean_in) / self.std_in
        elif data_type == "output":
            return (data - self.mean_out) / self.std_out

    def inverse_transform_output(self, data):
        return data * self.std_out + self.mean_out


### 🌍 Data Module: Loading, Normalization, and Splitting

This section handles the entire data pipeline, from loading the `.zarr` dataset to preparing PyTorch-ready DataLoaders.

#### `ClimateDataset`
- A simple PyTorch `Dataset` wrapper that preloads the entire (normalized) dataset into memory using Dask.
- Converts the data to PyTorch tensors and handles any `NaN` checks up front.

#### `ClimateDataModule`
A PyTorch Lightning `DataModule` that handles:
- ✅ **Loading data** from different SSP scenarios and ensemble members
- ✅ **Broadcasting non-spatial inputs** (like CO₂) to match spatial grid size
- ✅ **Normalization** using mean/std computed from training data only
- ✅ **Splitting** into training, validation, and test sets:
  - Training: All months from selected SSPs (except last 10 years of SSP370)
  - Validation: Last 10 years (120 months) of SSP370
  - Test: Last 10 years of SSP245 (unseen scenario)
- ✅ **Batching** and parallelized data loading via PyTorch `DataLoader`s
- ✅ **Latitude-based area weighting** for fair climate metric evaluation
- Shape of the inputs are Batch_Size X 5 (num_input_variables) X 48 X 72
- Shape of ouputputs are Batch_Size X 2 (num_output_variables) X 48 X 72

> ℹ️ **Note:** You likely won’t need to modify this class but feel free to make modifications if you want to inlcude different ensemble mebers to feed more data to your models


In [6]:
class ClimateDataset(Dataset):
    def __init__(self, inputs_dask, outputs_dask, output_is_normalized=True, seq_len=12, predict_seq=True):
        self.size = inputs_dask.shape[0] - seq_len + 1
        print(f"Creating dataset with {self.size} samples...")

        inputs_np = inputs_dask.compute()
        outputs_np = outputs_dask.compute()
        
        self.inputs = torch.from_numpy(inputs_np).float()
        self.outputs = torch.from_numpy(outputs_np).float()

        self.seq_len = seq_len
        self.predict_seq = predict_seq

        if torch.isnan(self.inputs).any() or torch.isnan(self.outputs).any():
            raise ValueError("NaNs found in dataset")

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        x = self.inputs[idx:idx+self.seq_len]      # [T, C, H, W]
        y = self.outputs[idx:idx+self.seq_len] if self.predict_seq else self.outputs[idx+self.seq_len-1]
        return x, y

class AugmentedDataset(Dataset):
    def __init__(self, base_dataset, transform):
        self.base = base_dataset
        self.transform = transform

    def __len__(self):
        return len(self.base)

    def __getitem__(self, idx):
        x, y = self.base[idx]
        x = self.transform(x)
        return x, y


class ClimateDataModule(pl.LightningDataModule):
    def __init__(
        self,
        path,
        input_vars,
        output_vars,
        train_ssps,
        test_ssp,
        target_member_id,
        val_split=0.1,
        test_months=120,
        batch_size=32,
        num_workers=0,
        seed=42,
        augmentations=None,
        seq_len=12, 
        predict_seq=True
    ):
        super().__init__()
        self.path = path
        self.input_vars = input_vars
        self.output_vars = output_vars
        self.train_ssps = train_ssps
        self.test_ssp = test_ssp
        self.target_member_id = target_member_id
        self.val_split = val_split
        self.test_months = test_months
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.seed = seed
        self.normalizer = Normalizer()
        self.augmentations=augmentations
        self.seq_len = seq_len
        self.predict_seq = predict_seq

    def prepare_data(self):
        assert os.path.exists(self.path), f"Data path not found: {self.path}"

    def setup(self, stage=None):
        ds = xr.open_zarr(self.path, consolidated=False, chunks={"time": 24})
        spatial_template = ds["rsdt"].isel(time=0, ssp=0, drop=True)

        def load_ssp(ssp, member_id=0):
            input_dask, output_dask = [], []
            for var in self.input_vars:
                da_var = ds[var].sel(ssp=ssp)
                if "latitude" in da_var.dims:
                    da_var = da_var.rename({"latitude": "y", "longitude": "x"})
                if "member_id" in da_var.dims:
                    # da_var = da_var.sel(member_id=self.target_member_id)
                    da_var = da_var.sel(member_id=member_id)
                if set(da_var.dims) == {"time"}:
                    da_var = da_var.broadcast_like(spatial_template).transpose("time", "y", "x")
                input_dask.append(da_var.data)

            for var in self.output_vars:
                # da_out = ds[var].sel(ssp=ssp, member_id=self.target_member_id)
                da_out = ds[var].sel(ssp=ssp, member_id=member_id)
                if "latitude" in da_out.dims:
                    da_out = da_out.rename({"latitude": "y", "longitude": "x"})
                output_dask.append(da_out.data)

            return da.stack(input_dask, axis=1), da.stack(output_dask, axis=1)

        train_input, train_output, val_input, val_output = [], [], None, None

        for ssp in self.train_ssps:
            for member_id in self.target_member_id:
                x, y = load_ssp(ssp, member_id=member_id)
                if ssp == "ssp370":
                    #val_input = x[-self.test_months:]
                    #val_output = y[-self.test_months:]
                    # train_input.append(x[:-self.test_months])
                    # train_output.append(y[:-self.test_months])
    
                    if member_id == 0:
                        val_input = x[-(self.test_months + self.seq_len - 1):]
                        val_output = y[-(self.test_months + self.seq_len - 1):]
                    train_input.append(x[:-self.test_months])
                    train_output.append(y[:-self.test_months])
                else:
                    train_input.append(x)
                    train_output.append(y)

        train_input = da.concatenate(train_input, axis=0)
        train_output = da.concatenate(train_output, axis=0)

        self.normalizer.set_input_statistics(
            mean=da.nanmean(train_input, axis=(0, 2, 3), keepdims=True).compute(),
            std=da.nanstd(train_input, axis=(0, 2, 3), keepdims=True).compute(),
        )
        self.normalizer.set_output_statistics(
            mean=da.nanmean(train_output, axis=(0, 2, 3), keepdims=True).compute(),
            std=da.nanstd(train_output, axis=(0, 2, 3), keepdims=True).compute(),
        )

        train_input_norm = self.normalizer.normalize(train_input, "input")
        train_output_norm = self.normalizer.normalize(train_output, "output")
        val_input_norm = self.normalizer.normalize(val_input, "input")
        val_output_norm = self.normalizer.normalize(val_output, "output")

        test_input, test_output = load_ssp(self.test_ssp)
        
        #test_input = test_input[-self.test_months:]
        #test_output = test_output[-self.test_months:]
        test_input = test_input[-(self.test_months + self.seq_len - 1):]  # Need full context
        test_output = test_output[-(self.test_months + self.seq_len - 1):]  # Only evaluating these months

        test_input_norm = self.normalizer.normalize(test_input, "input")

        train_dataset = ClimateDataset(train_input_norm, train_output_norm, seq_len=self.seq_len, predict_seq=self.predict_seq)
        if self.augmentations is not None:
            self.train_dataset = AugmentedDataset(train_dataset, self.augmentations)
        else:
            self.train_dataset = train_dataset
        self.val_dataset = ClimateDataset(val_input_norm, val_output_norm, seq_len=self.seq_len, predict_seq=self.predict_seq)
        self.test_dataset = ClimateDataset(test_input_norm, test_output, output_is_normalized=False, seq_len=self.seq_len, predict_seq=self.predict_seq)

        self.lat = spatial_template.y.values
        self.lon = spatial_template.x.values
        self.area_weights = xr.DataArray(get_lat_weights(self.lat), dims=["y"], coords={"y": self.lat})

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True,
                          num_workers=self.num_workers, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False,
                          num_workers=self.num_workers, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False,
                          num_workers=self.num_workers, pin_memory=True)

    def get_lat_weights(self):
        return self.area_weights

    def get_coords(self):
        return self.lat, self.lon

### ⚡ ClimateEmulationModule: Lightning Wrapper for Climate Model Emulation

This is the core model wrapper built with **PyTorch Lightning**, which organizes the training, validation, and testing logic for the climate emulation task. Lightning abstracts away much of the boilerplate code in PyTorch-based deep learning workflows, making it easier to scale models.

#### ✅ Key Features

- **`training_step` / `validation_step` / `test_step`**: Standard Lightning hooks for computing loss and predictions at each stage. The loss used is **Mean Squared Error (MSE)**.

- **Normalization-aware outputs**:
  - During validation and testing, predictions and targets are denormalized before evaluation using stored mean/std statistics.
  - This ensures evaluation is done in real-world units (Kelvin and mm/day).

- **Metric Evaluation** via `_evaluate()`:
  For each variable (`tas`, `pr`), it calculates:
  - **Monthly Area-Weighted RMSE**
  - **Time-Mean RMSE** (RMSE on 10-year average's)
  - **Time-Stddev MAE** (MAE on 10-year standard deviation; a measure of temporal variability)
    
  These metrics reflect the competition's evaluation criteria and are logged and printed.

- **Kaggle Submission Writer**:
  After testing, predictions are saved to a `.csv` file in the required Kaggle format via `_save_submission()`.

- **Saving Predictions for Visualization**:
  - Validation predictions are saved tao `val_preds.npy` and `val_trues.npy`
  - These can be loaded later for visual inspection of the model's performance.

 🔧 **Feel free to modify any part of this module** (loss functions, evaluation, training logic) to better suit your model or training pipeline / Use pure PyTorch etc.

⚠️ The **final submission `.csv` file must strictly follow the format and naming convention used in `_save_submission()`**, as these `ID`s are used to match predictions to the hidden test set during evaluation.



In [7]:
import pandas as pd

class ClimateEmulationModule(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-4):
        super().__init__()
        self.model = model
        self.save_hyperparameters(ignore=['model']) # Save all hyperparameters except the model to self.hparams.<param_name>
        self.criterion = nn.MSELoss()
        self.normalizer = None
        self.val_preds, self.val_targets = [], []
        self.test_preds, self.test_targets = [], []

    def forward(self, x):
        return self.model(x)

    def on_fit_start(self):
        self.normalizer = self.trainer.datamodule.normalizer  # Get the normalizer from the datamodule (see above)

    def training_step(self, batch, batch_idx):
        x, y = batch # Unpack inputs and targets (this is the output of the _getitem_ method in the Dataset above)
        y_hat = self(x)   # Forward pass
        loss = self.criterion(y_hat, y)  # Calculate loss
        self.log("train/loss", loss)  # Log loss for tracking
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val/loss", loss)

        y_hat_np = self.normalizer.inverse_transform_output(y_hat.detach().cpu().numpy())
        y_np = self.normalizer.inverse_transform_output(y.detach().cpu().numpy())

        """B, T, C, H, W = y_hat.shape

        y_hat_np = self.normalizer.inverse_transform_output(
            y_hat.detach().cpu().numpy().reshape(-1, C, H, W)
        ).reshape(B, T, C, H, W)
        
        y_np = self.normalizer.inverse_transform_output(
            y.detach().cpu().numpy().reshape(-1, C, H, W)
        ).reshape(B, T, C, H, W)"""

        
        
        self.val_preds.append(y_hat_np)
        self.val_targets.append(y_np)

        return loss

    def on_validation_epoch_end(self):
        # Concatenate all predictions and ground truths from each val step/batch into one array
        preds = np.concatenate(self.val_preds, axis=0)
        trues = np.concatenate(self.val_targets, axis=0)
        self._evaluate(preds, trues, phase="val")
        np.save("val_preds.npy", preds)
        np.save("val_trues.npy", trues)
        self.val_preds.clear()
        self.val_targets.clear()

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        y_hat_np = self.normalizer.inverse_transform_output(y_hat.detach().cpu().numpy())
        y_np = y.detach().cpu().numpy()

        """B, T, C, H, W = y_hat.shape

        y_hat_np = self.normalizer.inverse_transform_output(
            y_hat.detach().cpu().numpy().reshape(-1, C, H, W)
        ).reshape(B, T, C, H, W)
        
        #y_np = self.normalizer.inverse_transform_output(
        #    y.detach().cpu().numpy().reshape(-1, C, H, W)
        #).reshape(B, T, C, H, W)
        y_np = y.detach().cpu().numpy()"""
        
        self.test_preds.append(y_hat_np)
        self.test_targets.append(y_np)

    def on_test_epoch_end(self):
        # Concatenate all predictions and ground truths from each test step/batch into one array
        preds = np.concatenate(self.test_preds, axis=0)
        trues = np.concatenate(self.test_targets, axis=0)
        self._evaluate(preds, trues, phase="test")
        self._save_submission(preds)
        self.test_preds.clear()
        self.test_targets.clear()

    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=self.hparams.learning_rate, weight_decay=1e-2)

    def _evaluate(self, preds, trues, phase="val"):
        datamodule = self.trainer.datamodule
        area_weights = datamodule.get_lat_weights()
        lat, lon = datamodule.get_coords()
        output_vars = datamodule.output_vars

        # Flatten batch and time dims: [B, T, C, H, W] -> [B*T, C, H, W]
        #preds = preds.reshape(-1, preds.shape[2], preds.shape[3], preds.shape[4])
        #trues = trues.reshape(-1, trues.shape[2], trues.shape[3], trues.shape[4])
        time = np.arange(preds.shape[0])

        for i, var in enumerate(output_vars):
            p = preds[:, i]
            t = trues[:, i]
            p_xr = xr.DataArray(p, dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})
            t_xr = xr.DataArray(t, dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})

            # RMSE
            rmse = np.sqrt(((p_xr - t_xr) ** 2).weighted(area_weights).mean(("time", "y", "x")).item())
            # RMSE of time-mean
            mean_rmse = np.sqrt(((p_xr.mean("time") - t_xr.mean("time")) ** 2).weighted(area_weights).mean(("y", "x")).item())
            # MAE of time-stddev
            std_mae = np.abs(p_xr.std("time") - t_xr.std("time")).weighted(area_weights).mean(("y", "x")).item()

            print(f"[{phase.upper()}] {var}: RMSE={rmse:.4f}, Time-Mean RMSE={mean_rmse:.4f}, Time-Stddev MAE={std_mae:.4f}")
            self.log_dict({
                f"{phase}/{var}/rmse": rmse,
                f"{phase}/{var}/time_mean_rmse": mean_rmse,
                f"{phase}/{var}/time_std_mae": std_mae,
            })

    def _save_submission(self, predictions):
        datamodule = self.trainer.datamodule
        lat, lon = datamodule.get_coords()
        output_vars = datamodule.output_vars

        # Flatten batch and time dims: [B, T, C, H, W] -> [B*T, C, H, W]
        #predictions = predictions.reshape(-1, predictions.shape[2], predictions.shape[3], predictions.shape[4])
        
        # Only keep the last timestep of each sequence
        #predictions = predictions[:, -1]  # shape: [B, C, H, W]
        time = np.arange(predictions.shape[0])

        rows = []
        for t_idx, t in enumerate(time):
            for var_idx, var in enumerate(output_vars):
                for y_idx, y in enumerate(lat):
                    for x_idx, x in enumerate(lon):
                        row_id = f"t{t_idx:03d}_{var}_{y:.2f}_{x:.2f}"
                        pred = predictions[t_idx, var_idx, y_idx, x_idx]
                        rows.append({"ID": row_id, "Prediction": pred})

        df = pd.DataFrame(rows)
        os.makedirs("submissions", exist_ok=True)
        filepath = f"submissions/kaggle_submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df.to_csv(filepath, index=False)
        print(f"✅ Submission saved to: {filepath}")

### ⚡ Training & Evaluation with PyTorch Lightning

This block sets up and runs the training and testing pipeline using **PyTorch Lightning’s `Trainer`**, which abstracts away much of the boilerplate in deep learning workflows.

- **Modular Setup**:
  - `datamodule`: Handles loading, normalization, and batching of climate data.
  - `model`: A convolutional neural network that maps climate forcings to predicted outputs.
  - `lightning_module`: Wraps the model with training/validation/test logic and metric evaluation.

- **Trainer Flexibility**:
  The `Trainer` accepts a wide range of configuration options from `config["trainer"]`, including:
  - Number of epochs
  - Precision (e.g., 16-bit or 32-bit)
  - Device configuration (CPU, GPU, or TPU)
  - Determinism, logging, callbacks, and more

In [8]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(
        self,
        d_model=512,
        nhead=8,
        num_encoder_layers=6,
        num_decoder_layers=6,
        dim_feedforward=2048,
        dropout=0.1
    ):
        super().__init__()
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self._reset_parameters()
        self.d_model = d_model
        self.pos_encoder = PositionalEncoding(d_model, dropout)

    def _reset_parameters(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self,
                src,
                tgt,
                src_mask=None,
                tgt_mask=None,
                memory_mask=None):
        src = self.pos_encoder(src)
        tgt = self.pos_encoder(tgt)
        return self.transformer(src, tgt,
                                src_mask=src_mask,
                                tgt_mask=tgt_mask,
                                memory_mask=memory_mask)


class ClimateTransformer(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        d_model=128,
        nhead=8,
        num_layers=4,
        dim_feedforward=512,
        dropout=0.1,
    ):
        super().__init__()
        self.patch_embed = nn.Conv2d(in_channels, d_model, kernel_size=1)
        self.pos_encoder = PositionalEncoding(d_model, dropout=dropout)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )
        self.head = nn.Linear(d_model, out_channels)

        self.out_channels = out_channels

    def forward(self, x):
        x = self.patch_embed(x)
        B, C, H, W = x.shape
        x = x.flatten(2).permute(2, 0, 1)
        x = self.pos_encoder(x)
        x = self.transformer(x)

        x = x.permute(1, 0, 2)
        x = self.head(x)
        x = x.permute(0, 2, 1)
        x = x.view(B, self.out_channels, H, W)
        return x

In [9]:
class Relative2DPositionalEncoding(nn.Module):
    def __init__(self, num_rows: int, num_cols: int, d_model: int):
        super().__init__()
        assert d_model % 2 == 0, "d_model must be even"
        self.num_rows = num_rows
        self.num_cols = num_cols
        self.d_model = d_model
        d_half = d_model // 2

        self.row_embed = nn.Parameter(torch.randn(num_rows, d_half))
        self.col_embed = nn.Parameter(torch.randn(num_cols, d_half))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: [seq_len, B, d_model] where seq_len = num_rows * num_cols
        """
        device = x.device
        row_embed = self.row_embed.to(device)
        col_embed = self.col_embed.to(device)

        pe = torch.cat([
            row_embed[:, None, :].expand(self.num_rows, self.num_cols, -1),
            col_embed[None, :, :].expand(self.num_rows, self.num_cols, -1)
        ], dim=-1).reshape(-1, self.d_model)  # [seq_len, d_model]

        return x + pe.unsqueeze(1)  # [seq_len, B, d_model]


class HierarchicalPatchTransformer(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        d_model: int = 128,
        nhead: int = 8,
        depths: tuple[int, int] = (2, 2),
        patch_sizes: tuple[int, int] = (4, 2),
        dim_feedforward: int = 256,
        dropout: float = 0.1,
        height: int = 48,
        width: int = 72
    ):
        super().__init__()
        p1, p2 = patch_sizes
        H1, W1 = height // p1, width // p1
        H2, W2 = H1 // p2, W1 // p2
        self.d_model = d_model

        self.patch_embed1 = nn.Conv2d(in_channels, d_model, kernel_size=p1, stride=p1)
        self.pos1 = Relative2DPositionalEncoding(H1, W1, d_model)
        enc1 = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, activation='gelu')
        self.encoder1 = nn.TransformerEncoder(enc1, num_layers=depths[0])

        self.patch_embed2 = nn.Conv2d(d_model, d_model, kernel_size=p2, stride=p2)
        self.pos2 = Relative2DPositionalEncoding(H2, W2, d_model)
        enc2 = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, activation='gelu')
        self.encoder2 = nn.TransformerEncoder(enc2, num_layers=depths[1])

        self.up1 = nn.ConvTranspose2d(d_model, d_model, kernel_size=p2, stride=p2)
        self.up2 = nn.ConvTranspose2d(d_model, out_channels, kernel_size=p1, stride=p1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, C, H, W = x.shape
        p1 = self.patch_embed1.kernel_size[0]
        p2 = self.patch_embed2.kernel_size[0]

        x1 = self.patch_embed1(x)
        seq1 = x1.flatten(2).permute(2, 0, 1)
        seq1 = self.pos1(seq1)
        seq1 = self.encoder1(seq1)
        x1 = seq1.permute(1, 2, 0).view(B, self.d_model, H // p1, W // p1)

        x2 = self.patch_embed2(x1)
        seq2 = x2.flatten(2).permute(2, 0, 1)
        seq2 = self.pos2(seq2)
        seq2 = self.encoder2(seq2)
        x2 = seq2.permute(1, 2, 0).view(B, self.d_model, H // (p1*p2), W // (p1*p2))

        u1 = self.up1(x2)
        out = self.up2(u1)
        return out


In [10]:
class TemporalHierarchicalTransformer(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        seq_len: int = 12,
        d_model: int = 128,
        nhead: int = 8,
        depths: tuple[int, int] = (2, 2),
        patch_sizes: tuple[int, int] = (4, 2),
        dim_feedforward: int = 256,
        dropout: float = 0.1,
        height: int = 48,
        width: int = 72
    ):
        super().__init__()
        self.seq_len = seq_len
        self.d_model = d_model
        p1, p2 = patch_sizes
        H1, W1 = height // p1, width // p1
        H2, W2 = H1 // p2, W1 // p2
        self.spatial_size = H2 * W2
        self.H2, self.W2 = H2, W2

        # Shared patch encoder (like before)
        self.patch_embed1 = nn.Conv2d(in_channels, d_model, kernel_size=p1, stride=p1)
        self.patch_embed2 = nn.Conv2d(d_model, d_model, kernel_size=p2, stride=p2)

        # Positional encodings
        self.temporal_pos = nn.Parameter(torch.randn(seq_len, 1, d_model))
        self.spatial_pos = nn.Parameter(torch.randn(H2 * W2, d_model))

        # Transformer encoder
        enc = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, activation='gelu')
        self.transformer = nn.TransformerEncoder(enc, num_layers=sum(depths))

        # Decoder
        self.up1 = nn.ConvTranspose2d(d_model, d_model, kernel_size=p2, stride=p2)
        self.up2 = nn.ConvTranspose2d(d_model, out_channels, kernel_size=p1, stride=p1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: [B, T, C, H, W]
        B, T, C, H, W = x.shape
        p1 = self.patch_embed1.kernel_size[0]
        p2 = self.patch_embed2.kernel_size[0]
        #H2, W2 = H // (p1 * p2), W // (p1 * p2)
        H2, W2 = self.H2, self.W2

        # Step 1: apply shared spatial patching per timestep
        x = x.view(B * T, C, H, W)
        x = self.patch_embed1(x)
        x = self.patch_embed2(x)  # [B*T, d_model, H2, W2]
        x = x.view(B, T, self.d_model, H2, W2)

        # Step 2: flatten spatial dims and add positional encodings
        x = x.permute(0, 1, 3, 4, 2).reshape(B, T * H2 * W2, self.d_model)  # [B, T*P, d_model]

        # Temporal-spatial positional encoding
        t_pos = self.temporal_pos[:T].expand(-1, H2 * W2, -1).reshape(T * H2 * W2, 1, self.d_model)
        s_pos = self.spatial_pos[:H2 * W2].repeat(T, 1).reshape(T * H2 * W2, self.d_model)
        x = x.permute(1, 0, 2)  # [T*P, B, d_model]
        x = x + t_pos.to(x.device) + s_pos.unsqueeze(1).to(x.device)

        # Step 3: Transformer encoding
        x = self.transformer(x)  # [T*P, B, d_model]

        # Step 4: reshape and decode per timestep
        """
        x = x.permute(1, 0, 2).reshape(B, T, H2, W2, self.d_model)
        x = x.permute(0, 1, 4, 2, 3).reshape(B * T, self.d_model, H2, W2)

        # Step 5: upsample
        x = self.up1(x)
        x = self.up2(x)
        x = x.view(B, T, -1, H, W)  # [B, T, 2, H, W]
        return x
        """

        x = x.permute(1, 0, 2)
        x = x.view(B, T, H2 * W2, self.d_model)
        x_last = x[:, -1]  # [B, H2*W2, d_model]
        x_last = x_last.reshape(B, self.d_model, H2, W2)

        x = self.up1(x_last)
        x = self.up2(x)
        return x  # [B, 2, H, W]




In [11]:
import torch
import torch.nn as nn


class TemporalHierarchicalTransformer(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        seq_len: int = 12,
        d_model: int = 128,
        nhead: int = 8,
        depths: tuple[int, int] = (2, 2),
        patch_sizes: tuple[int, int] = (4, 2),
        dim_feedforward: int = 256,
        dropout: float = 0.1,
        height: int = 48,
        width: int = 72
    ):
        super().__init__()
        self.seq_len = seq_len
        self.d_model = d_model
        p1, p2 = patch_sizes
        H1, W1 = height // p1, width // p1
        H2, W2 = H1 // p2, W1 // p2
        self.spatial_size = H2 * W2
        self.H2, self.W2 = H2, W2
        self.H1, self.W1 = H1, H2

        # ----------------------------------------
        # 1) Shared patch encoder + LayerNorm
        # ----------------------------------------
        # First conv: in_channels → d_model, patch size = p1
        self.patch_embed1 = nn.Conv2d(in_channels, d_model, kernel_size=p1, stride=p1)
        # Immediately apply LayerNorm over flattened “(H1*W1)×d_model” tokens
        self.patch_norm1 = nn.LayerNorm(d_model)

        # Second conv: d_model → d_model, patch size = p2
        self.patch_embed2 = nn.Conv2d(d_model, d_model, kernel_size=p2, stride=p2)
        # Then normalize those “(H2*W2)×d_model” tokens
        self.patch_norm2 = nn.LayerNorm(d_model)

        # ----------------------------------------
        # 2) Positional encodings (learnable)
        # ----------------------------------------
        self.temporal_pos = nn.Parameter(torch.randn(seq_len, 1, d_model))
        self.spatial_pos  = nn.Parameter(torch.randn(H2 * W2, d_model))

        # ----------------------------------------
        # 3) TransformerEncoder (pre-norm)
        #    using norm_first=True for a “pre-norm” block
        # ----------------------------------------
        # Build one layer with norm_first=True (pre-norm)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='gelu',
            norm_first=True
        )
        # Stack `sum(depths)` of these
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=sum(depths))

        # ----------------------------------------
        # 4) Decoder / Upsampling
        # ----------------------------------------
        # First upsample from (H2, W2) back to (H1, W1)
        self.up1 = nn.ConvTranspose2d(d_model, d_model, kernel_size=p2, stride=p2)
        # Then upsample from (H1, W1) back to (H, W)
        self.up2 = nn.ConvTranspose2d(d_model, out_channels, kernel_size=p1, stride=p1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Input:
            x: [B, T, C, H, W]
        Output:
            out: [B, out_channels, H, W]
            
        (We predict only the last frame’s output.)
        """
        B, T, C, H, W = x.shape
        H1, W1 = self.H1, self.W1
        H2, W2 = self.H2, self.W2
        p1 = self.patch_embed1.kernel_size[0]  # first patch size
        p2 = self.patch_embed2.kernel_size[0]  # second patch size

        # ----------------------------------------
        # Step 1: Shared spatial patching per timestep
        # ----------------------------------------
        # Merge batch & time dims: [B*T, C, H, W]
        x = x.view(B * T, C, H, W)

        # 1.1 First Conv → [B*T, d_model, H1, W1]
        x = self.patch_embed1(x)
        # Flatten spatial → [B*T, d_model, H1*W1], then transpose → [B*T, H1*W1, d_model]
        x = x.flatten(2).transpose(1, 2)
        # Apply LayerNorm across the last dimension (d_model)
        x = self.patch_norm1(x)
        # Restore to [B*T, d_model, H1, W1]
        x = x.transpose(1, 2).view(B * T, self.d_model, H1, W1)

        # 1.2 Second Conv → [B*T, d_model, H2, W2]
        x = self.patch_embed2(x)
        # Flatten again → [B*T, d_model, H2*W2] → transpose → [B*T, H2*W2, d_model]
        x = x.flatten(2).transpose(1, 2)
        # Apply LayerNorm again
        x = self.patch_norm2(x)
        # Reshape to [B, T, d_model, H2, W2]
        x = x.transpose(1, 2).view(B, T, self.d_model, H2, W2)

        # ----------------------------------------
        # Step 2: Flatten spatial dims & add positional encodings
        # ----------------------------------------
        # Permute so tokens are contiguous: [B, T, H2, W2, d_model]
        x = x.permute(0, 1, 3, 4, 2).reshape(B, T * H2 * W2, self.d_model)
        # Now x is [B, (T·H2·W2), d_model]

        # Build temporal + spatial positional biases:
        #  - temporal_pos: [T, 1, d_model] → expand over H2*W2 → [T, H2*W2, d_model]
        t_pos = self.temporal_pos[:T]                       # [T, 1, d_model]
        t_pos = t_pos.expand(-1, H2 * W2, -1)               # [T, H2*W2, d_model]
        t_pos = t_pos.reshape(T * H2 * W2, 1, self.d_model) # [T·P, 1, d_model]

        #  - spatial_pos: [H2*W2, d_model] → repeat for each t → [T·(H2·W2), d_model]
        s_pos = self.spatial_pos[: H2 * W2]                 # [P, d_model]
        s_pos = s_pos.repeat(T, 1).reshape(T * H2 * W2, self.d_model)  # [T·P, d_model]

        # Permute x → [T·P, B, d_model] for Transformer
        x = x.permute(1, 0, 2)  # (seq_len = T·P,  B,  d_model)
        # Add both positional terms
        x = x + t_pos.to(x.device) + s_pos.unsqueeze(1).to(x.device)

        # ----------------------------------------
        # Step 3: TransformerEncoder (pre-norm)
        # ----------------------------------------
        # Now x stays [T·P, B, d_model] throughout.
        x = self.transformer(x)  # → [T·P, B, d_model]

        # ----------------------------------------
        # Step 4: Reshape & take only “last‐frame” tokens
        # ----------------------------------------
        # Bring back to [B, T·P, d_model]
        x = x.permute(1, 0, 2).reshape(B, T, H2 * W2, self.d_model)
        # We only decode the last timestep (index = T-1)
        x_last = x[:, -1]                # [B, H2·W2, d_model]
        x_last = x_last.reshape(B, self.d_model, H2, W2)

        # ----------------------------------------
        # Step 5: Upsample back to original H×W
        # ----------------------------------------
        x_out = self.up1(x_last)         # [B, d_model, H1, W1]
        x_out = self.up2(x_out)          # [B, out_channels, H, W]

        return x_out


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TemporalHierarchicalTransformerPlus(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        seq_len: int = 12,
        d_model: int = 128,
        nhead: int = 8,
        depths: tuple[int,int] = (2,2),
        patch_sizes: tuple[int,int] = (4,2),
        dim_feedforward: int = 256,
        dropout: float = 0.1,
        height: int = 48,
        width: int = 72,
    ):
        super().__init__()
        self.seq_len = seq_len
        self.d_model = d_model

        # 1) patch sizes and spatial dims
        p1, p2 = patch_sizes
        H1, W1 = height // p1, width // p1
        H2, W2 = H1 // p2, W1 // p2
        self.H2, self.W2 = H2, W2
        self.spatial_size = H2 * W2

        # 2) spatial patch embed
        self.patch_embed1 = nn.Conv2d(in_channels, d_model, kernel_size=p1, stride=p1)
        self.patch_embed2 = nn.Conv2d(d_model, d_model, kernel_size=p2, stride=p2)

        # 3) temporal conv stem
        self.temporal_conv = nn.Conv1d(d_model, d_model, kernel_size=3, padding=1, bias=False)
        self.temporal_bn   = nn.BatchNorm1d(d_model)

        # 4) positional encodings
        self.temporal_pos = nn.Parameter(torch.randn(seq_len, 1, d_model))
        self.spatial_pos  = nn.Parameter(torch.randn(H2 * W2, d_model))

        # 5) transformer
        total_layers      = sum(depths)
        encoder_layer     = nn.TransformerEncoderLayer(
            d_model, nhead, dim_feedforward, dropout, activation='gelu'
        )
        self.transformer  = nn.TransformerEncoder(encoder_layer, num_layers=total_layers)
        self.trans_norm   = nn.LayerNorm(d_model)

        # 6) decoder up-modules + skip projections
        self.up1        = nn.ConvTranspose2d(d_model, d_model, kernel_size=p2, stride=p2)
        self.up2        = nn.ConvTranspose2d(d_model, out_channels, kernel_size=p1, stride=p1)
        self.skip_proj1 = nn.ConvTranspose2d(d_model, d_model, kernel_size=p2, stride=p2)
        self.skip_proj2 = nn.ConvTranspose2d(d_model, d_model, kernel_size=p1, stride=p1)
        self.skip_out   = nn.Conv2d(d_model, out_channels, kernel_size=1, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, T, C, H, W = x.shape
        assert T == self.seq_len, f"Expected T={self.seq_len}, got {T}"
        p1 = self.patch_embed1.kernel_size[0]
        p2 = self.patch_embed2.kernel_size[0]
        H2, W2 = self.H2, self.W2
        d = self.d_model

        # A) apply patch_embed1 + patch_embed2
        z1 = x.view(B * T, C, H, W)
        z2 = self.patch_embed1(z1)
        z3 = self.patch_embed2(z2)
        z4 = z3.view(B, T, d, H2, W2)

        # B) skip_early
        skip_early = z4[:, 0].clone()

        # C) temporal‐conv stem
        stem = z4.permute(0, 3, 4, 2, 1).contiguous()
        stem = stem.view(B * H2 * W2, d, T)
        stem = self.temporal_conv(stem)
        stem = self.temporal_bn(stem)
        stem = F.gelu(stem)
        stem = stem.view(B, H2, W2, d, T).permute(0, 4, 3, 1, 2).contiguous()
        x3 = stem

        # D) flatten + positional encoding
        x4 = x3.permute(0, 1, 3, 4, 2).reshape(B, T * (H2 * W2), d)
        x4 = x4.permute(1, 0, 2).contiguous()
        tp = self.temporal_pos[:T].expand(-1, H2*W2, -1).reshape(T*H2*W2, 1, d)
        sp = self.spatial_pos[:H2*W2].repeat(T, 1).reshape(T*H2*W2, d).unsqueeze(1)
        x4 = x4 + tp.to(x4.device) + sp.to(x4.device)

        # E) transformer + residual + LayerNorm
        tr_out = self.transformer(x4)
        alpha = 0.5
        x5 = x4 + alpha * tr_out
        x5 = self.trans_norm(x5)

        # F) reshape back to [B,T,d,H2,W2]
        x5 = x5.permute(1, 0, 2).contiguous()
        x5 = x5.view(B, T, H2, W2, d)
        x5 = x5.permute(0, 1, 4, 2, 3).contiguous()

        # G) last time slice
        x_last = x5[:, -1]

        # H) decoder up1 + skip fusion
        u1 = self.up1(x_last)
        se = self.skip_proj1(skip_early)
        u1 = u1 + se
        u1 = F.gelu(u1)

        # I) final up2 + long skip
        u2 = self.up2(u1)
        se2 = self.skip_proj1(skip_early)
        se2 = F.gelu(se2)
        se2 = self.skip_proj2(se2)
        se2 = F.gelu(se2)
        se2 = self.skip_out(se2)

        out = u2 + se2
        return out


In [32]:
datamodule = ClimateDataModule(**config["data"])
"""model = HierarchicalPatchTransformer(
    in_channels=len(config["data"]["input_vars"]),
    out_channels=len(config["data"]["output_vars"]),
    dropout=0.1,
)"""

model = TemporalHierarchicalTransformerPlus(
    in_channels=len(config["data"]["input_vars"]),
    out_channels=len(config["data"]["output_vars"]),
    dropout=0.1,
    seq_len=config['data']['seq_len'],
    patch_sizes = (2,4),

)

lightning_module = ClimateEmulationModule(model, learning_rate=1e-3)

trainer = pl.Trainer(**config["trainer"])
trainer.fit(lightning_module, datamodule=datamodule)   # Training

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


Creating dataset with 8098 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name      | Type                                | Params | Mode 
--------------------------------------------------------------------------
0 | model     | TemporalHierarchicalTransformerPlus | 1.4 M  | train
1 | criterion | MSELoss                             | 0      | train
--------------------------------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.778     Total estimated model params size (MB)
54        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

[VAL] tas: RMSE=1.3324, Time-Mean RMSE=0.5861, Time-Stddev MAE=0.3166
[VAL] pr: RMSE=1.9356, Time-Mean RMSE=0.2452, Time-Stddev MAE=0.7424


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.


[VAL] tas: RMSE=1.2584, Time-Mean RMSE=0.4467, Time-Stddev MAE=0.2525
[VAL] pr: RMSE=1.9263, Time-Mean RMSE=0.2071, Time-Stddev MAE=0.7228


# Test model

**IMPORTANT:** Please note that the test metrics will be bad because the test targets have been corrupted on the public Kaggle dataset.
The purpose of testing below is to generate the Kaggle submission file based on your model's predictions, which you can submit to the competition.

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

# 1) Switch to eval and pull out model + dataloader
lightning_module.eval()
model = lightning_module.model  # your ClimateTransformer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

train_loader = datamodule.train_dataloader()
errors = []

# 2) Loop and record per-sample error + raw arrays
with torch.no_grad():
    for idx, batch in enumerate(train_loader):
        x, y = batch[0].to(device), batch[1].to(device)
        y_hat = model(x)
        # compute MSE map, then mean over channels + spatial dims
        mse_map = F.mse_loss(y_hat, y, reduction='none')      # [B,C,H,W]
        per_sample = mse_map.mean(dim=[1,2,3])                 # [B]
        for b in range(x.size(0)):
            errors.append({
                'err': per_sample[b].item(),
                'x'  : x[b].cpu().numpy(),     # [C,H,W]
                'y'  : y[b].cpu().numpy(),
                'ŷ' : y_hat[b].cpu().numpy()
            })

# 3) Pick top-K
K = 3
errors = sorted(errors, key=lambda d: d['err'], reverse=True)[:K]

# 4) Plot them
fig, axes = plt.subplots(K, 3, figsize=(12, 4*K), constrained_layout=True)

for i, sample in enumerate(errors):
    x, y_true, y_pred, e = sample['x'], sample['y'], sample['ŷ'], sample['err']
    # collapse multi-channel by mean:
    err_map = np.mean((y_pred - y_true)**2, axis=0)  # [H,W]

    # show channel 0 of input & truth (or pick whichever)
    axes[i,0].imshow(x[0], cmap='viridis')
    axes[i,0].set_title(f"Input #{i} (err={e:.3f})")
    axes[i,0].axis('off')

    axes[i,1].imshow(y_true[0], cmap='viridis')
    axes[i,1].set_title("Ground Truth")
    axes[i,1].axis('off')

    im = axes[i,2].imshow(err_map, cmap='magma')
    axes[i,2].set_title("Per-pixel MSE")
    axes[i,2].axis('off')
    fig.colorbar(im, ax=axes[i,2], fraction=0.046, pad=0.04)

plt.show()


In [33]:
trainer.test(lightning_module, datamodule=datamodule) 

trainer.save_checkpoint("transformer_v16.ckpt")

Creating dataset with 8098 samples...
Creating dataset with 360 samples...
Creating dataset with 360 samples...


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

[TEST] tas: RMSE=290.5651, Time-Mean RMSE=290.5244, Time-Stddev MAE=3.5054
[TEST] pr: RMSE=4.3292, Time-Mean RMSE=3.8230, Time-Stddev MAE=1.4320
✅ Submission saved to: submissions/kaggle_submission_20250602_040500.csv


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/pr/rmse        │    4.3292436599731445     │
│  test/pr/time_mean_rmse   │    3.8229527473449707     │
│   test/pr/time_std_mae    │    1.4320029020309448     │
│       test/tas/rmse       │     290.5650939941406     │
│  test/tas/time_mean_rmse  │     290.5243835449219     │
│   test/tas/time_std_mae   │     3.505403995513916     │
└───────────────────────────┴───────────────────────────┘

### Plotting Utils


In [ ]:
def plot_comparison(true_xr, pred_xr, title, cmap='viridis', diff_cmap='RdBu_r', metric=None):
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))

    vmin = min(true_xr.min().item(), pred_xr.min().item())
    vmax = max(true_xr.max().item(), pred_xr.max().item())

    # Ground truth
    true_xr.plot(ax=axs[0], cmap=cmap, vmin=vmin, vmax=vmax, add_colorbar=True)
    axs[0].set_title(f"{title} (Ground Truth)")

    # Prediction
    pred_xr.plot(ax=axs[1], cmap=cmap, vmin=vmin, vmax=vmax, add_colorbar=True)
    axs[1].set_title(f"{title} (Prediction)")

    # Difference
    diff = pred_xr - true_xr
    abs_max = np.max(np.abs(diff))
    diff.plot(ax=axs[2], cmap=diff_cmap, vmin=-abs_max, vmax=abs_max, add_colorbar=True)
    axs[2].set_title(f"{title} (Difference) {f'- {metric:.4f}' if metric else ''}")

    plt.tight_layout()
    plt.show()


### 🖼️ Visualizing Validation Predictions

This cell loads saved validation predictions and compares them to the ground truth using spatial plots. These visualizations help you qualitatively assess your model's performance.

For each output variable (`tas`, `pr`), we visualize:

- **📈 Time-Mean Map**: The 10-year average spatial pattern for both prediction and ground truth. Helps identify long-term biases or spatial shifts.
- **📊 Time-Stddev Map**: Shows the standard deviation across time for each grid cell — useful for assessing how well the model captures **temporal variability** at each location.
- **🕓 Random Timestep Sample**: Visual comparison of prediction vs ground truth for a single month. Useful for spotting fine-grained anomalies or errors in specific months.

> These plots provide intuition beyond metrics and are useful for debugging spatial or temporal model failures.


In [ ]:
# Load validation predictions
# make sure to have run the validation loop at least once
val_preds = np.load("val_preds.npy")
val_trues = np.load("val_trues.npy")

lat, lon = datamodule.get_coords()
output_vars = datamodule.output_vars
time = np.arange(val_preds.shape[0])

for i, var in enumerate(output_vars):
    pred_xr = xr.DataArray(val_preds[:, i], dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})
    true_xr = xr.DataArray(val_trues[:, i], dims=["time", "y", "x"], coords={"time": time, "y": lat, "x": lon})

    # --- Time Mean ---
    plot_comparison(true_xr.mean("time"), pred_xr.mean("time"), f"{var} Val Time-Mean")

    # --- Time Stddev ---
    plot_comparison(true_xr.std("time"), pred_xr.std("time"), f"{var} Val Time-Stddev", cmap="plasma")

    # --- Random timestep ---
    t_idx = np.random.randint(0, len(time))
    plot_comparison(true_xr.isel(time=t_idx), pred_xr.isel(time=t_idx), f"{var} Val Sample Timestep {t_idx}")


## 🧪 Final Notes

This notebook is meant to serve as a **baseline template** — a starting point to help you get up and running quickly with the climate emulation challenge.

You are **not** required to stick to this exact setup. In fact, we **encourage** you to:

- 🔁 Build on top of the provided `DataModule`. 
- 🧠 Use your own model architectures or training pipelines that you’re more comfortable with 
- ⚗️ Experiment with ideas  
- 🥇 Compete creatively to climb the Kaggle leaderboard  
- 🙌 Most importantly: **have fun** and **learn as much as you can** along the way

This challenge simulates a real-world scientific problem, and there’s no single "correct" approach — so be curious, experiment boldly, and make it your own!


# Load And Test Pretrained Model

In [ ]:
datamodule2 = ClimateDataModule(**config["data"])

model2 = TemporalHierarchicalTransformer(
    in_channels=len(config["data"]["input_vars"]),
    out_channels=len(config["data"]["output_vars"]),
    seq_len=config["data"]["seq_len"],
    depths=(2, 2),
    patch_sizes=(4, 2),
    dropout=0.1,
    height=48,
    width=72,
)


restored = ClimateEmulationModule.load_from_checkpoint(
    "/kaggle/working/transformer_v4.ckpt", # change to path where checkpoint is stored
    model=model2,
    learning_rate=config["training"]["lr"],
)

restored.eval()
restored.freeze()
model2 = restored.model

trainer2 = pl.Trainer(**config["trainer"])

restored.normalizer = datamodule2.normalizer

trainer2.test(restored, datamodule=datamodule2)